<a href="https://colab.research.google.com/github/benardt/genealogyKPI/blob/main/genealogy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genealogy

## Configuration file

In [ ]:
# configuration data

my_config = {
    'login': 'xxxx',
    'password': 'xxxx',
    'login_page': 'https://www.geneanet.org/connexion/',
}


## Modules and dependencies

In [ ]:
%%capture
!apt-get update
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install ./google-chrome-stable_current_amd64.deb
!echo $PATH
!google-chrome --product-version
!pip install seleniumbase

In [ ]:
import math
import numpy as np
import pandas as pd
import io, zipfile, re

from bs4 import BeautifulSoup

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# code linter

!pip install pycodestyle
!pip install --index-url https://test.pypi.org/simple/ nbpep8

from nbpep8.nbpep8 import pep8

# Add pep8(_ih) at the end of the code cell to see PEP8 analysis.

## Main class

In [ ]:
class Ancestors:
    """     Global class    """

    def get_polar_coor(self, sosa):
        """Sosa number to coordinates

        returns the coordinates of an individual according to his sosa number.

        Args:
            param1 (int): The sosa number.

        Returns:
            tuple (float, float): x and y coordinates.

        """
        generation = int(np.log2(sosa))
        n_total = 2**generation - 2**(generation-1)
        phi = (sosa-2**generation) * np.pi / n_total
        return generation * np.cos(phi), generation * np.sin(phi)

    def bfs(self, node):
        """Build family graph with BFS algorithm

        Use Breadth-first search (BFS) to find all sosa ancestors
        from node and fom graph

        @param: str  - node: starting node id (de-cujus)
        """
        # visited = []   # List to keep track of visited nodes.
        # visited.append(node)
        queue = []     # Initialize a queue
        queue.append(node)
        sosas = []
        sosas.append(1) # root sosa is sosa = 1

        rel = []
        li = {}

        while queue:
            child = queue.pop(0)
            sosa = sosas.pop(0)
            rel.append(sosa)
            li[sosa] = child

            # id = 0 or 1
            #  + index=0 is sir (ie man)
            #  + index=1 is dam (ie woman)
            for id, parent in enumerate(self.id_graph[child]):
                # visited is not used because :
                #   - a parent can have several child
                #   - a parent can be multiple ancestor
                # if parent not in visited:
                #     visited.append(parent)
                if parent != None:
                    queue.append(parent)
                    sosas.append(2 * sosa + id)

        return (rel, li)

    def soup_parser(self, soup):
        """Parse soup data got from geneanet.org

        Args:
            param1 (soup): complete web page

        """
        texts = []
        id_childs = []
        id_with_noname = []

        all_lis = soup.find_all("li")

        for li0 in all_lis:
            if "Génération" in li0.text:
                ul = li0.find("ul")
                for li in ul.find_all("li"):
                    # remove white space separator thousands
                    string = li.text.replace(u'\xa0', u'')
                    texts.append(string)


        # Get all individuals by sosa
        for string in texts:
            if re.match('^[0-9]+\s-\s\?\s\?$', string):
                noname = re.search(r'^([0-9]+)\s-\s\?\s\?$', string).group(1)
                id_with_noname.append(int(noname))
            else:
                if re.match('^[0-9]+\s-\s', string):
                    sosa = re.search(r'^([0-9]+)\s-\s', string).group(1)
                    nom = re.search(r'^[0-9]+\s-\s(.*)', string).group(1)
                    self.ids.append(int(sosa))
                    id_childs.append(int(sosa))
                    self.id_names[int(sosa)] = nom
                if re.match('^[0-9]+\s=>\s[0-9]+$', string):
                    sosa = re.search(r'^([0-9]+)\s=>\s[0-9]+$', string).group(1)
                    sosa1 = re.search(r'^[0-9]+\s=>\s([0-9]+)$', string).group(1)
                    self.ids.append(int(sosa))
                    id_childs.append(int(sosa1))

        # Build graph - dict child:[sir,dam]
        for sosa, child in zip(self.ids, id_childs):
            # work only with sosa in child[]
            if sosa in id_childs:
                # add parent if parent exist in sosas[]
                husb = id_childs[self.ids.index(sosa*2)  ] if sosa*2   in self.ids else None
                wife = id_childs[self.ids.index(sosa*2+1)] if sosa*2+1 in self.ids else None
                self.id_graph[sosa] = [husb, wife]

        # Remove parent with name '? ?'
        for key in self.id_graph:
            for idx, parent in enumerate(self.id_graph[key]):
                if parent in id_with_noname:
                    self.id_graph[key][idx] = None


    def file_parser(self, file_string):
        """Parse GEDCOM file

        """

        file_array = file_string.splitlines(True)

        # get family
        pattern_indi = "^0\s@I[0-9]+@\sINDI$"
        pattern_name = "^1\sNAME\s"
        pattern_sexe = "^1\sSEX\s"
        pattern_fam = "^0\s@F[0-9]+@\sFAM$"
        pattern_chil = "^1 CHIL @I[0-9]+@$"
        pattern_husb = "^1 HUSB @I[0-9]+@$"
        pattern_wife = "^1 WIFE @I[0-9]+@$"
        chi = []
        sir = []
        dam = []

        # Get all families
        for string in file_array:
            if re.match(pattern_fam, string):
                husb, wife = None, None
            if re.match(pattern_husb, string):
                husb = re.search(r'^1 HUSB @I([0-9]+)@$', string).group(1)
            if re.match(pattern_wife, string):
                wife = re.search(r'^1 WIFE @I([0-9]+)@$', string).group(1)
            if re.match(pattern_chil, string):
                chil = re.search(r'^1 CHIL @I([0-9]+)@$', string).group(1) 
                chi.append(int(chil))
                sir.append(int(husb) if husb != None else None)
                dam.append(int(wife) if wife != None else None)

        # get all individuals
        is_indi = 1
        for string in file_array:
            if re.match(r'^0\s', string) and is_indi == 0:
                self.ids.append(id)
                self.id_names[id] = nom
                is_indi = 1
            if re.match(pattern_indi, string):
                id = int(re.search(r'^0\s@I([0-9]+)@\sINDI$', string).group(1))
                nom = ""
                is_indi = 0
            if re.match(pattern_name, string):
                nom = re.search(r'^1\sNAME\s(.*)$', string).group(1)


        # Add father and mother to all individuals
        for id in self.ids:
            if id in chi:
                idx = chi.index(id)
                self.id_graph[id] = [sir[idx], dam[idx]]
            else:
                self.id_graph[id] = [None, None]



    def __init__(self, soup=None, file_string='', mode='connect'):

        self.ids = []
        # dict - id (int): name (str)
        self.id_names = {}
        # dict - child: [sir, dam]
        self.id_graph = {}

        if mode == "connect":
            print('=> Geneanet parser')
            if soup: 
                self.soup_parser(soup)
            else:
                print("Error: No data")
        elif mode == "file":
            print('=> File parser')
            self.file_parser(file_string)
        else:
            print('Parser error')
        
        # ids
        # id_names
        # id_graph

    def getName(self, sosa):
        """

        return name of person identified by sosa number

        @param: int - sosa number
        @return: str - name of sosa
        """
        
        if sosa != 0:
            ids = self.sosa2ids[sosa]
            name = self.id_names[ids]
        else:
            name = "None"
        return name

    def build_sosas(self, souche_id):
        """

        Build 4 lists:
            sosas{}:          dict - all sosas (with implex) and unique sosas
            count[]:          list - number of occurence for individual (implex)

        """

        self.sosas = {'all': [], 'unique_only': [], 'unique_all': []}

        self.sosas['all'], self.sosa2ids = self.bfs(souche_id)

        sosas = list(self.sosa2ids.keys())
        ids = list(self.sosa2ids.values())
        
        all_id = [self.sosa2ids[sosa] for sosa in self.sosas['all']]
        unique_id = list(dict.fromkeys(all_id))
        self.sosas['unique_only'] = [sosas[ids.index(id)] for id in unique_id]

        self.sosas['unique_all'] = [sosas[ids.index(id)] for id in all_id]

        self.count = []
        for sosa in self.sosas['all']:
            id = self.sosa2ids[sosa]
            self.count.append(ids.count(id))


    def build_parents(self):
        """

        Convert id_graph{} dict - {child: [sir, dam], ...} to list of list
        [sosa_child1, sosa_child2, ...]
        [[], [sosa_sir2, sosa_dam2], [sosa_sir3], ...]

        """
        # id_graph{} dict - child: [sir, dam]

        sosas = list(self.sosa2ids.keys())
        ids = list(self.sosa2ids.values())

        self.parents = [[] for _ in  self.sosas['unique_only']]

        for idx, sosa in enumerate(self.sosas['unique_only']):
            id = self.sosa2ids[sosa]
            for parent in self.id_graph[id]:
                if parent != None:
                    self.parents[idx] += [sosas[ids.index(parent)]]

    def inbreeding(self,n=4):
        """Compute inbreeding coefficients

        Compute inbreeding coefficient for all individuals from parents data.
        Two individuals can have the same parents. This is the starting point
        for implex.

        Returns:
            numpy matrix: inbreedings matrix
            dataframe: all datas

        """

        nb = len(self.parents)
        print("Number of individuals to be processed: ", nb)

        # List of list of parents (convert sosa to index)
        P = [[nb-1-self.sosas['unique_only'].index(parent) for parent in parents] for parents in reversed(self.parents)]

        print("Parents converted")

        A = np.diag(nb * [1.0])

        for (i, j), _ in np.ndenumerate(A):
            if i == j:
                if len(P[j]) == 2:
                    [id_sir, id_dam] = P[j]
                    A[i, j] += 0.5 * A[id_sir, id_dam]
            elif i < j:
                for idx in P[j]:
                    A[i, j] += 0.5 * A[i, idx]
                    A[j, i] = A [i, j]


        # flip matrix A to have youngest at bottom and left
        A = np.flip(A)
        print('First step done!')

        # inbreeding coefficients are on diagonal of matrix A : 100 * (value - 1)
        INBREEDINGS = 100 * (A.diagonal() - 1)

        df = pd.DataFrame({
            "x": [self.get_polar_coor(sosa)[0] for sosa in self.sosas['unique_only']],
            "y": [self.get_polar_coor(sosa)[1] for sosa in self.sosas['unique_only']],
            "parents": self.parents,
            "sosa": self.sosas['unique_only'],
            "name": [self.getName(sosa) + "<br>" + str(inbreed) for sosa, inbreed in zip(self.sosas['unique_only'],INBREEDINGS)],
            "inbreeding": INBREEDINGS,
            "inbreeding_color": [np.log(inbreeding+0.00001) for inbreeding in INBREEDINGS]
            })

        return A - np.flip(np.diag(nb * [1.0])), df


## Geneanet scrapping

In [ ]:
# Get data by connecting on geneanet.org website

from seleniumbase import SB

with SB(browser="chrome", chromium_arg="--no-sandbox, --disable-dev-shm-usage") as sb:
    sb.open(my_config['login_page'])
    sb.type("#_username", my_config['login'])
    sb.type("#_password", my_config['password'])
    sb.click("#_submit")
    sb.open("https://www.geneanet.org/")
    suffixe = '&m=A&t=N&v=50&lang=fr'
    r2 = sb.get_page_source()
    soup2 = BeautifulSoup(r2, "html.parser")
    td_tag_list = soup2.find_all("a", attrs={"gaq-event": "show-souche"})
    print('https:'+td_tag_list[0]['href']+suffixe)
    sb.open('https:'+td_tag_list[0]['href']+suffixe)
    r = sb.get_page_source()

soup = BeautifulSoup(r, "html.parser")

# Souche de l'arbre

td_tag_list = soup2.find_all("a", attrs={"gaq-event": "show-souche"}) 
print(td_tag_list[0]['href'])

## Instanciation and code for vizualization

In [ ]:
# Get data from local file GEDCOM (zip)
# Paser of GEDCOM file

archive = zipfile.ZipFile('/content/drive/MyDrive/data/benardt_2023-01-09.zip', 'r')
file_data = archive.read('base.ged')
file_string = file_data.decode("utf-8")

#mya = Ancestors(file_string=file_string, mode='file')

# Get data by connecting to Geneanet websit

mya = Ancestors(soup, mode='connect')


# Build sosas
#mya.build_sosas(288)
mya.build_sosas(1)
mya.build_parents()

print("Number of individuals: ", len(mya.id_graph))

print(len(mya.sosas['unique_only']), mya.sosas['unique_only'])
print(len(mya.parents), mya.parents)

print(len(mya.sosas['all']), mya.sosas['all'])
print(len(mya.sosas['unique_all']), mya.sosas['unique_all'])
print(len(mya.count), mya.count)
print(len(mya.sosas['unique_only']))

print(mya.sosas['all'][2470:2550])

for idx in range(11):
    print(mya.sosas['all'][idx], mya.id_names[mya.sosa2ids[mya.sosas['all'][idx]]])

In [ ]:
import networkx as nx

G = nx.DiGraph()
G.add_nodes_from(mya.sosas['unique_only'])

for idx, parents in enumerate(mya.parents):
    for parent in parents:
        G.add_edge(mya.sosas['unique_only'][idx], parent)

print(nx.degree(G)) 
print(nx.density(G))

pos=nx.spring_layout(G)
print(pos)

In [ ]:
H, dfi = mya.inbreeding(200)

# Figure
fig = make_subplots(rows=1, cols=2, column_titles=["Inbreeding coefficient for each individual [log scale]", "Matrix"])
fig.add_trace(go.Heatmap(z=H),
    row=1, col=2
)

fig.add_trace(go.Scatter(x=dfi["x"], y=dfi["y"], mode='markers',
    marker=dict(color=dfi["inbreeding_color"], size=4, colorscale="Viridis_r"),
    text=dfi["name"]),
    row=1, col=1  
)

fig.update_layout(
    yaxis=dict(scaleanchor = "x", scaleratio = 1),
    yaxis2=dict(scaleanchor = "x2", scaleratio = 1),
    )

fig.show()

In [ ]:


def get_all_childs(sosa_target):
    """
    get all the descendants (who exits) of an individual
    @param: int - sosa
    @return: list - all sosas of descendants
    """

    idxs = [idx for idx, val in enumerate(mya.sosas['unique_all']) if val == sosa_target]

    family = []
    sosas_identical = []
    for idx in idxs:
        sosa = mya.sosas['all'][idx]
        sosas_identical.append(sosa)
        family.append(sosa)
        child = int(sosa/2)
        # All sosas are ancestors.
        # So end (of while loop) is sosa = 1 [de cujus]
        while child != 1:
            family.append(child)
            child = int(child/2)

    return family,sosas_identical

sosa_target = 278944
#sosa_target = 266385
#sosa_target = 266370
#sosa_target = 33296

descendants, identicals = get_all_childs(sosa_target)

def wrap2lines(word):
    '''
    Wrap in line on 2 lines with <br> separator

    @param: str
    @return: str
    '''
    perfect_idx = int(len(word)/2)
    possible_idxs = [pos for pos, char in enumerate(word) if char == ' ']
    final_idx = min(possible_idxs, key=lambda x: abs(x-perfect_idx))
    return word[:final_idx] + '<br>' + word[final_idx:]

# build DataFrame to display scatter plot with all sosas

dfp2 = pd.DataFrame({
    "X": [mya.get_polar_coor(sosa)[0] for sosa in mya.sosas['all']],
    "Y": [mya.get_polar_coor(sosa)[1] for sosa in mya.sosas['all']],
    "U": ["unique" if sosa in mya.sosas['unique_all'] else "not unique" for sosa in mya.sosas['all']],
    "G": [int(math.log2(sosa)) for sosa in mya.sosas['all']],
    "R": [sosa for sosa in mya.sosas['all']],
    "S": [sosa for sosa in mya.sosas['all']],
    "Sbin": [bin(sosa) for sosa in mya.sosas['all']],
    "Sun_L": [wrap2lines(str(sosa) + " - " + mya.getName(sosa)) for sosa in mya.sosas['all']],
    "Sun_P": [wrap2lines(str(int(sosa/2)) + " - " + mya.getName(int(sosa/2))) for sosa in mya.sosas['all']],
    "Sun_V": [360/(2**int(math.log2(sosa))) for sosa in mya.sosas['all']],
    "Child": [int(sosa/2) for sosa in mya.sosas['all']],
    "V": [360/(2**int(math.log2(sosa))) for sosa in mya.sosas['all']],
    "T": ["sosa: " + str(sosa) + " / " + mya.getName(sosa) + "<br>" + str(c) for sosa,c in zip(mya.sosas['all'],mya.count)],
    "C": [color for color in mya.count]
    })

dfp3 = pd.DataFrame({
    "X": [mya.get_polar_coor(sosa)[0] for sosa in descendants],
    "Y": [mya.get_polar_coor(sosa)[1] for sosa in descendants],
    "T": ["sosa: " + str(sosa) + " / " + mya.getName(sosa) for sosa in descendants],
    "C": [1] * len(descendants)
    })

dfp4 = pd.DataFrame({
    "X": [mya.get_polar_coor(sosa)[0] for sosa in identicals],
    "Y": [mya.get_polar_coor(sosa)[1] for sosa in identicals],
    "T": ["sosa: " + str(sosa) + " / " + mya.getName(sosa) for sosa in identicals],
    "C": [0] * len(identicals)
    })

fig = make_subplots(rows=1, cols=2, column_titles=["All descendants of "+ mya.getName(sosa_target)+" [" + str(sosa_target) +"]", "Number of times a sosa is an ancestor"])

fig.add_trace(
    go.Scatter(x=dfp2["X"][dfp2["U"]=="unique"], y=dfp2["Y"][dfp2["U"]=="unique"], mode='markers', text=dfp2["T"][dfp2["U"]=="unique"],
                name="all unique sosas"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=dfp2["X"], y=dfp2["Y"], mode='markers', text=dfp2["T"],
                   marker=dict(color=dfp2["C"]),
               name="all sosas"),
    
    row=1, col=2
)

# Affiche en sur-impression (par dessus) tous les descendants de "sosa_target"
fig.add_trace(
    go.Scatter(x=dfp3["X"], y=dfp3["Y"], mode='markers', text=dfp3["T"],
                    name="descendants",
                   marker=dict(color=dfp3["C"])),
    row=1, col=1
)

# Affiche en sur-impression (par dessus) tous les descendants de "sosa_target"
fig.add_trace(
    go.Scatter(x=dfp4["X"], y=dfp4["Y"], mode='markers', text=dfp4["T"],
                    name="identical",
                   marker=dict(color=dfp4["C"],colorscale="Viridis_r")),
    row=1, col=1
)

fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )


In [ ]:

# build DataFrame to display scatter plot with all sosas
dfp = pd.DataFrame({
    "X": [mya.get_polar_coor(sosa)[0] for sosa in mya.sosas['all']],
    "Y": [mya.get_polar_coor(sosa)[1] for sosa in mya.sosas['all']],
    "Generation": [int(math.log2(sosa)) for sosa in mya.sosas['all']],
    "Name": [mya.getName(sosa) for sosa in mya.sosas['unique_all']],
    "Sosa": mya.sosas['all'],
    "Nb of sosas": mya.count,
    "Unique sosa": mya.sosas['unique_all']
    })

# convert column in str to get palette color (not linear color)
dfp["Nb of sosas"] = dfp["Nb of sosas"].astype(str)
fig = px.scatter(data_frame=dfp, x="X", y="Y",color="Nb of sosas",
                 hover_data=['Name', 'Sosa', 'Unique sosa', 'Generation'],
                 width=800, height=800,
                 title="Number of times a sosa is an ancestor [non linear color]")
fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
fig.show()

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets


def draw_sun(gene,root,df):
    idxs = []
    for idx, _ in enumerate(mya.sosas['all']):
        if df['G'][idx] < gene and df['Sbin'][idx].startswith(bin(root)):
            idxs.append(idx)

    fig = go.Figure(go.Sunburst(
        labels=df['Sun_L'][idxs][1:],
        parents=df['Sun_P'][idxs][1:],
        values=df['Sun_V'][idxs][1:],
        branchvalues="total"
    ))
    fig.update_layout(margin=dict(t=0, l=0, r=0, b=0))
    fig.show()

myw1 = widgets.BoundedIntText(
    value=4,
    min=0,
    max=50000000,
    step=1,
    description='Sosa:',
    disabled=False
)

myw2 = widgets.IntSlider(
    value=7,
    min=0,
    max=30,
    step=1,
    description='Génération:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

@interact
def make_fig(generation=myw2, root=myw1):
    return draw_sun(generation, root, dfp2)



In [ ]:

U, C, G, N, S = [], [], [], [], []

for sosa in mya.sosas['unique_only']:
    idx = mya.sosas['all'].index(sosa)
    c = mya.count[idx]
    if c > 2:
        U.append(sosa)
        C.append(c)
        G.append(int(math.log2(sosa)))
        N.append(mya.getName(sosa))
        idxs = [idx for idx, val in enumerate(mya.sosas['unique_all']) if val == sosa]
        S.append([mya.sosas['all'][idx] for idx in idxs])

newdf = pd.DataFrame({
    "sosa": U,
    "generation": G,
    "count": C,
    "name": N,
    "sosas": S
    })

fig = make_subplots(
    rows=1, cols=1,
    specs=[[{"type": "table"}]]
)

fig.add_trace(
    go.Table(
        header=dict(
            values=["sosa", "generation", "count", "name", "sosas"],
            align="center"
        ),
        cells=dict(
            values=[newdf[k].tolist() for k in newdf.columns],
            align="left"),
        columnwidth=[1,1,1,3,8]
    ),
    row=1, col=1
)

fig.update_layout(
    height=600,
    showlegend=False,
    title_text="Most several anecesters",
)

fig.show()


In [ ]:

generations, gene_tot = [], []
Th, T, U, C, G = [], [], [], [], []

for sosa in mya.sosas['unique_only']:
    generations.append(int(np.log2(sosa)))

for sosa in mya.sosas['all']:
    gene_tot.append(int(np.log2(sosa)))

G = sorted(dfp2['G'].unique())

for generation in G:
    Th.append(2**generation)
    U.append(generations.count(generation))
    T.append(gene_tot.count(generation))
    C.append(int(10000*gene_tot.count(generation)/(2**generation))/100)

newdf = pd.DataFrame({
    "generation": G,
    "unique nb": U,
    "total nb": T,
    "completion": C,
    "theoritical nb": Th
    })

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "xy"}, {"type": "violin"}], [{"type": "table"}, {"type": "sunburst"}]]
)

fig.add_trace(
    go.Table(
        header=dict(
            values=["generation", "unique nb", "total nb", "completion", "theoritical nb"],
            align="center"
        ),
        cells=dict(
            values=[newdf[k].tolist() for k in newdf.columns],
            align="left"),
        columnwidth=[1, 1, 1, 1, 1]
    ),
    row=2, col=1
)

fig.update_layout(
    height=600,
    showlegend=False,
    title_text="Data by generation"
)

fig.add_trace(go.Bar(name='Total', x=T, y=G, orientation='h'), row=1, col=1)
fig.add_trace(go.Bar(name='Unique', x=U, y=G, orientation='h'), row=1, col=1)
fig.add_trace(go.Scatter(name='Completition', x=C, y=G, xaxis="x2"), row=1, col=1)

fig.add_trace(go.Violin(y=dfp2["G"][dfp2['U'] == 'unique'],
                        side='negative', name="Unique",
                        line_color='blue', x0="G"), row=1, col=2)
fig.add_trace(go.Violin(y=dfp2["G"],
                        side='positive', name="Total",
                        line_color='orange', x0="G"), row=1, col=2)

G = 9

fig.add_trace(go.Sunburst(
    labels=dfp2['S'][dfp2['G'] < G][1:],
    parents=dfp2['Child'][dfp2['G'] < G][1:],
    values=dfp2['V'][dfp2['G'] < G][1:],
    branchvalues="total"
), row=2, col=2)

fig.update_xaxes(title="people #", row=1, col=1)
fig.update_yaxes(title="generation #", row=1, col=1)
fig.update_yaxes(title="generation #", row=1, col=2)

fig.update_layout(
    height=700,
    width=900,
    showlegend=True,
    title_text="Global charts"
)

fig.show()


#Bibliography

Georgelis, A. (2018). Multiperspective visualization of genealogy data.
https://www.diva-portal.org/smash/get/diva2:1242034/FULLTEXT01.pdf



Ball, R., & Cook, D. (2014, February). A family-centric genealogy visualization paradigm. In Proceedings of 14th Annual Family History Technology Workshop.
http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.589.6435&rep=rep1&type=pdf


Köhle, D. Spatio-Temporal Genealogy Visualization with WorldLines.
https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.645.7667&rep=rep1&type=pdf



 http://www.aviz.fr/geneaquilts



 Calculation of inbreeding and relationship, the tabular method http://www.ihh.kvl.dk/htm/kc/popgen/genetics/4/5.htm

B Tier. Computing inbreeding coefficients quickly. Genetics Selection Evolution, 1990, 22 (4), pp.419-430. hal-00893856 https://hal.archives-ouvertes.fr/hal-00893856/

